<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import multiprocessing
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# # Descargar la carpeta de dataset
# import os
# import platform
# if os.access('./songs_dataset', os.F_OK) is False:
#     if os.access('songs_dataset.zip', os.F_OK) is False:
#         if platform.system() == 'Windows':
#             !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
#         else:
#             !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
#     !unzip -q songs_dataset.zip   
# else:
#     print("El dataset ya se encuentra descargado")

In [ ]:
sys.exit(0)

import requests
import zipfile

url = "https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip"
response = requests.get(url)
with open("songs_dataset.zip", "wb") as file:
    file.write(response.content)

with zipfile.ZipFile("songs_dataset.zip", "r") as zip_ref:
    zip_ref.extractall(".")

In [2]:
# Posibles bandas
os.listdir("./songs_dataset/")

['adele.txt',
 'al-green.txt',
 'alicia-keys.txt',
 'amy-winehouse.txt',
 'beatles.txt',
 'bieber.txt',
 'bjork.txt',
 'blink-182.txt',
 'bob-dylan.txt',
 'bob-marley.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'bruno-mars.txt',
 'cake.txt',
 'dickinson.txt',
 'disney.txt',
 'dj-khaled.txt',
 'dolly-parton.txt',
 'dr-seuss.txt',
 'drake.txt',
 'eminem.txt',
 'janisjoplin.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'joni-mitchell.txt',
 'kanye-west.txt',
 'kanye.txt',
 'Kanye_West.txt',
 'lady-gaga.txt',
 'leonard-cohen.txt',
 'lil-wayne.txt',
 'Lil_Wayne.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'ludacris.txt',
 'michael-jackson.txt',
 'missy-elliott.txt',
 'nickelback.txt',
 'nicki-minaj.txt',
 'nirvana.txt',
 'notorious-big.txt',
 'notorious_big.txt',
 'nursery_rhymes.txt',
 'patti-smith.txt',
 'paul-simon.txt',
 'prince.txt',
 'r-kelly.txt',
 'radiohead.txt',
 'rihanna.txt']

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_13412\3849064916.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [5]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

Those are not tokens properly, but segmented sentences instead.

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 108762.3046875
Loss after epoch 1: 65536.0546875
Loss after epoch 2: 65764.6875
Loss after epoch 3: 65382.734375
Loss after epoch 4: 63376.90625
Loss after epoch 5: 63168.875
Loss after epoch 6: 62341.3125
Loss after epoch 7: 62222.5
Loss after epoch 8: 59677.75
Loss after epoch 9: 57449.0625
Loss after epoch 10: 56911.5625
Loss after epoch 11: 56068.75
Loss after epoch 12: 54925.875
Loss after epoch 13: 53780.625
Loss after epoch 14: 53130.4375
Loss after epoch 15: 53233.6875
Loss after epoch 16: 52228.75
Loss after epoch 17: 47708.5
Loss after epoch 18: 47391.625
Loss after epoch 19: 46832.75


(156986, 287740)

### 4 - Ensayar

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('sleep', 0.840125322341919),
 ('pretty', 0.8363893628120422),
 ('cry', 0.8120163083076477),
 ('seems', 0.7844411134719849),
 ('help', 0.7741866111755371),
 ('do', 0.7473866939544678),
 ('oh', 0.734216034412384),
 ('high', 0.7296016216278076),
 ('peace', 0.7280288934707642),
 ('try', 0.7220621109008789)]

In [14]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('our', -0.17201298475265503),
 ('shake', -0.181864932179451),
 ('six', -0.20115654170513153),
 ('yellow', -0.2037947028875351),
 ('four', -0.2073374092578888),
 ('five', -0.21242646872997284),
 ('seven', -0.22835353016853333),
 ('work', -0.23679497838020325),
 ('baby', -0.2485644668340683),
 ('going', -0.2526545822620392)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9642793536186218),
 ('three', 0.9451873302459717),
 ('seven', 0.9446370601654053),
 ('six', 0.9436722993850708),
 ('two', 0.9358287453651428),
 ('sixty', 0.9028005003929138),
 ('us', 0.7871828079223633),
 ('crying', 0.7871068120002747),
 ('said', 0.7658377885818481),
 ('each', 0.7559351325035095)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9205241799354553),
 ('buy', 0.9110379815101624),
 ('much', 0.8718225359916687),
 ('hide', 0.8277009725570679),
 ('just', 0.810702919960022)]

In [17]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [18]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.32544598  0.62910974 -0.20950606 -0.3848609  -0.44128808 -0.32321727
  0.8101613   0.46863794 -0.745866   -0.10273854  0.26681593 -0.83493966
  0.76483226  0.27383453 -0.52410686  0.59179413  0.22701955  0.27971926
 -0.73931557 -0.3853295   0.47279572  0.25792563  0.898064    0.24473208
 -0.33784464  0.02560429  0.36900985 -0.05037174 -0.27353403 -0.10801231
 -0.5588118  -0.34851873 -0.38177413 -0.6244563   0.12035489 -0.48409218
  0.7540877   0.04570149  0.4031569  -0.1824202   0.21544878  0.1782253
  0.22774968 -0.3037861   0.2668184   0.02464673  0.28484452  0.24105422
  0.5536561   0.16217443]


In [19]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9090447425842285),
 ('someone', 0.8905414938926697),
 ("didn't", 0.8402394652366638),
 ('need', 0.8379504680633545),
 ("there's", 0.8312346339225769),
 ('hope', 0.8263214230537415),
 ('somebody', 0.8244156837463379),
 ('nothing', 0.8202620148658752),
 ('buy', 0.8189709186553955)]

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9090447425842285),
 ('someone', 0.8905414938926697),
 ("didn't", 0.8402395248413086),
 ('need', 0.8379504680633545),
 ("there's", 0.8312346339225769),
 ('hope', 0.8263214230537415),
 ('somebody', 0.8244156837463379),
 ('nothing', 0.8202620148658752),
 ('buy', 0.8189709186553955),
 ('whoa', 0.8157616853713989)]

### 5 - Visualizar agrupación de vectores

In [21]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [23]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # renderer="colab" esto para plotly en colab

In [25]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # renderer="colab" esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.